In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp # 200 is success
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j # JSON 

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [4]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [5]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp # still response, but data could be total garbage 

<Response [200]>

In [6]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [7]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [8]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2380\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2010\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 651\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 191"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [56]:
# def call_api(list, df):

#     for x in list: 

#         ## 1.1 Query to pull some data 
#         example_naep_query = (
#             'https://www.nationsreportcard.gov/'
#             'Dataservice/GetAdhocData.aspx?'
#             'type=data&subject=writing&grade=' + str(x) + '&'
#             'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
    
#         ## use requests to call the api
#         naep_resp = requests.get(example_naep_query)
      
#         ## get the json contents of the response 
#         ## here, we're assuming valid response
#         naep_resp_j = naep_resp.json()
        
#         ## with result, turn it into a dataframe
#         naep_resp_d = pd.DataFrame(naep_resp_j['result'])

#         naep_resp_d = pd.concat([df, naep_resp_d])
        
#     return naep_resp_d

# df = None
# grades = ["4", "8", "12"]
# grades = ["8", "12"]
# #grades = ["12"]

# call_api(grades, df) 


### MING'S SOLUTION 

def get_grades(grades): 

    result = pd.DataFrame()
    
    for grade in grades: 
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        naep_resp = requests.get(example_naep_query)
        
        # try: 
        #     naep_resp_j = naep_resp.json()
        #     naep_resp_j
        #     naep_resp_d = pd.DataFrame(naep_resp_j['result'])
        #     result = pd.concat([result,naep_resp_d]) 

        try: 
            naep_resp_j = naep_resp.json()
            naep_resp_j
            naep_resp_d = pd.DataFrame(naep_resp_j['result'])
            result = pd.concat([result,naep_resp_d]) 
        
        except Exception as e:
            print(e)

    return result

grades = ['4','8','12']
get_grades(grades)


### process 5000 vs. 13,000 words 
# for i in range(5): 
#     x = get_data(i) 

Invalid control character at: line 1 column 289 (char 288)


,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [57]:
## get the key
API_KEY = "NDFqHKvO09kuhRdC3e58qgUsXpDEejlukDPoltRrNvPC4gnkgfEbtr-IZIph72GaCNXadr2BAw0eNN1RwHv8kBB4ss_rwL2W70YNqjHCi6-Rp-XmNs_dBfEF12UaZ3Yx"

In [59]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [60]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=sLbDg__Yx-ug3dVFokXa2A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sLbDg__Yx-ug3dVFokXa2A',
 'review_count': 252,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.4,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442,
 'business_hours': [{'open': [{'is_overnight': False,

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes
0,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,252,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,545,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
2,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,False,https://www.yelp.com/biz/duende-hanover-2?adju...,15,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",3.7,"{'latitude': 43.70074272457673, 'longitude': -...",[],NaN,"{'address1': '15 Lebanon St', 'address2': '', ...",+16033064826,(603) 306-4826,107.388862,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
3,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,29,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.1,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
4,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788,"[{'open': [{'is_overnight': True, 'start': '00...","{'business_temp_closed': None, 'menu_url': Non..."


In [61]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
qxm3VNmD0O2zw8m9U8SxZg,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,https://www.yelp.com/biz/duende-hanover-2?adju...,NaN,+16033064826,(603) 306-4826
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
wyV_NfYn4ZOfp_sHMDPcAw,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,https://www.yelp.com/biz/bistro-at-six-hanover...,$$,+16036430600,(603) 643-0600
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [63]:
# your code here

## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Boxford,MA,01921"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()

## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()


<Response [200]>

{'id': '9lgmabEjEtY39UeEkRYxwg',
 'alias': 'grove-at-briar-barn-inn-rowley',
 'name': 'Grove at Briar Barn Inn ',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3YnVD38CANXnVCxLKIQO8g/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/grove-at-briar-barn-inn-rowley?adjust_creative=sLbDg__Yx-ug3dVFokXa2A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sLbDg__Yx-ug3dVFokXa2A',
 'review_count': 111,
 'categories': [{'alias': 'newamerican', 'title': 'New American'},
  {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
  {'alias': 'diners', 'title': 'Diners'}],
 'rating': 4.3,
 'coordinates': {'latitude': 42.712186, 'longitude': -70.880364},
 'transactions': [],
 'location': {'address1': '101 Main St',
  'address2': '',
  'address3': None,
  'city': 'Rowley',
  'zip_code': '01969',
  'country': 'US',
  'state': 'MA',
  'display_address': ['101 Main St', 'Rowley, MA 01969']},
 'phone': '+19784845166',
 'display_phone': '(978) 484-5166',
 '

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,business_hours,attributes,price
0,9lgmabEjEtY39UeEkRYxwg,grove-at-briar-barn-inn-rowley,Grove at Briar Barn Inn,https://s3-media2.fl.yelpcdn.com/bphoto/3YnVD3...,False,https://www.yelp.com/biz/grove-at-briar-barn-i...,111,"[{'alias': 'newamerican', 'title': 'New Americ...",4.3,"{'latitude': 42.712186, 'longitude': -70.880364}",[],"{'address1': '101 Main St', 'address2': '', 'a...",+19784845166,(978) 484-5166,9393.194494,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",NaN
1,FU-KgLwZUcaxzdlbnVzViw,off-the-vine-tuscan-grille-rowley-2,Off the Vine Tuscan Grille,https://s3-media1.fl.yelpcdn.com/bphoto/sHmBTx...,False,https://www.yelp.com/biz/off-the-vine-tuscan-g...,127,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.8,"{'latitude': 42.703517, 'longitude': -70.910505}",[],"{'address1': '150 Newburyport Tpke', 'address2...",+19789488788,(978) 948-8788,6822.345505,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': '',...",$$
2,yDwaFVHpz0mLRtJ0laEnPA,cafe-sarina-georgetown,Cafe Sarina,https://s3-media3.fl.yelpcdn.com/bphoto/sqm5cY...,False,https://www.yelp.com/biz/cafe-sarina-georgetow...,53,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.2,"{'latitude': 42.705124363921485, 'longitude': ...",[],"{'address1': '269A Central St', 'address2': No...",+19787695012,(978) 769-5012,4451.810396,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht...",NaN
3,bt6hlLEkI0ZdXR8DTYPX2Q,the-spot-georgetown-2,The Spot,https://s3-media3.fl.yelpcdn.com/bphoto/r50oxp...,False,https://www.yelp.com/biz/the-spot-georgetown-2...,235,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.4,"{'latitude': 42.72503, 'longitude': -70.992153}",[],"{'address1': '19 W Main St', 'address2': '', '...",+19783523351,(978) 352-3351,6652.825798,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",$$
4,L2Kw0Q4c29Z0p77VkWvsfA,west-village-provisions-boxford,West Village Provisions,https://s3-media2.fl.yelpcdn.com/bphoto/HnzR8w...,False,https://www.yelp.com/biz/west-village-provisio...,29,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.8,"{'latitude': 42.7071724, 'longitude': -71.0638...",[pickup],"{'address1': '561 Main St', 'address2': '', 'a...",+19783529711,(978) 352-9711,8443.789989,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht...",$


In [64]:
import requests

url = "https://api.yelp.com/v3/businesses/bt6hlLEkI0ZdXR8DTYPX2Q/reviews?limit=20&sort_by=yelp_sort"

headers = {
    "accept": "application/json",
    "Authorization": "NDFqHKvO09kuhRdC3e58qgUsXpDEejlukDPoltRrNvPC4gnkgfEbtr-IZIph72GaCNXadr2BAw0eNN1RwHv8kBB4ss_rwL2W70YNqjHCi6-Rp-XmNs_dBfEF12UaZ3Yx"
}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "VALIDATION_ERROR", "description": "'NDFqHKvO09kuhRdC3e58qgUsXpDEejlukDPoltRrNvPC4gnkgfEbtr-IZIph72GaCNXadr2BAw0eNN1RwHv8kBB4ss_rwL2W70YNqjHCi6-Rp-XmNs_dBfEF12UaZ3Yx' does not match '^(?i)Bearer [A-Za-z0-9\\\\-\\\\_]{128}$'", "field": "Authorization", "instance": "NDFqHKvO09kuhRdC3e58qgUsXpDEejlukDPoltRrNvPC4gnkgfEbtr-IZIph72GaCNXadr2BAw0eNN1RwHv8kBB4ss_rwL2W70YNqjHCi6-Rp-XmNs_dBfEF12UaZ3Yx"}}
